In [1]:
pip install mlxtend

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2021/SampleSubmissionWiDS2021.csv
/kaggle/input/widsdatathon2021/SolutionTemplateWiDS2021.csv
/kaggle/input/widsdatathon2021/DataDictionaryWiDS2021.csv
/kaggle/input/widsdatathon2021/UnlabeledWiDS2021.csv
/kaggle/input/widsdatathon2021/TrainingWiDS2021.csv


In [2]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn import metrics
import seaborn as sns
import itertools
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics

# Classifiers
from sklearn.svm import NuSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingCVClassifier # <- Here is our boy

from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [3]:
import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor

from sklearn.metrics import mean_squared_error, classification_report
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

# import xgboost
# import lightgbm as lgb
# from lightgbm import LGBMClassifier

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import gc
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
pd.set_option('max_rows', 300)
import re

/kaggle/input/widsdatathon2021/SampleSubmissionWiDS2021.csv
/kaggle/input/widsdatathon2021/SolutionTemplateWiDS2021.csv
/kaggle/input/widsdatathon2021/DataDictionaryWiDS2021.csv
/kaggle/input/widsdatathon2021/UnlabeledWiDS2021.csv
/kaggle/input/widsdatathon2021/TrainingWiDS2021.csv


In [4]:
pd.set_option('display.max_columns', 300)
np.random.seed(566)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_colwidth', -1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [5]:
df1 =pd.read_csv('/kaggle/input/widsdatathon2021/TrainingWiDS2021.csv')
df2 =pd.read_csv('/kaggle/input/widsdatathon2021/UnlabeledWiDS2021.csv')

In [6]:
y=df1['diabetes_mellitus']

In [7]:
df = df1.drop(['diabetes_mellitus'], axis=1).append(df2)
df.shape

(140391, 180)

In [8]:
df.drop(df.columns[df.columns.str.contains('^h1')], axis=1, inplace=True)
df.drop(['hospital_id'], axis=1, inplace=True)
df.drop(df.columns[df.columns.str.contains('^Un')], axis=1, inplace=True)
df.drop(df.columns[df.columns.str.contains('invasive')], axis=1, inplace=True)
df.drop(df.columns[df.columns.str.contains('noninvasive')], axis=1, inplace=True)
df.drop(['icu_id'], axis=1, inplace=True)
cols = df.columns[df.isnull().mean()>0.7]
print(cols)

Index(['fio2_apache', 'paco2_apache', 'paco2_for_ph_apache', 'pao2_apache', 'ph_apache', 'd1_lactate_max', 'd1_lactate_min', 'd1_pao2fio2ratio_max', 'd1_pao2fio2ratio_min'], dtype='object')


In [9]:
df.drop(cols, axis=1, inplace=True)
df.drop(['readmission_status'], axis=1,inplace=True)
df.drop(['hepatic_failure'], axis=1,inplace=True)
threshold = 0.0
df.drop(df.std()[df.std()==threshold].index.values, axis=1,inplace=True)
df.shape

(140391, 90)

In [10]:
df['pp']=df['d1_sysbp_max']-df['d1_diasbp_max']
df['gcs']=df['gcs_eyes_apache']+df['gcs_motor_apache']+df['gcs_verbal_apache']
df['bmi_calc'] = df['weight']/(df['height']/100)**2
df.drop(['bmi'],axis=1,inplace=True)
df.shape

(140391, 92)

In [11]:
df['gcs']=df['gcs']+df['gcs_unable_apache']
df.drop(['gcs_eyes_apache'],axis=1,inplace=True)
df.drop(['gcs_motor_apache'],axis=1,inplace=True)
df.drop(['gcs_verbal_apache'],axis=1,inplace=True)
df.drop(['gcs_unable_apache'],axis=1,inplace=True)
df.shape

(140391, 88)

In [12]:
df.shape

(140391, 88)

In [13]:
df.head(3)

,encounter_id,age,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_max,d1_diasbp_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_temp_max,d1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,aids,cirrhosis,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,pp,gcs,bmi_calc
0,214826,68.00,0,Caucasian,M,180.30,Floor,Floor,admit,CTICU,0.54,73.90,2.30,113.00,502.01,0,0,0.40,31.00,2.51,168.00,118.00,27.40,0,40.00,36.00,134.00,39.30,NaN,0,14.10,68.00,37.00,119.00,72.00,89.00,46.00,34.00,10.00,100.00,74.00,131.00,73.00,39.90,37.20,2.30,2.30,0.40,0.40,31.00,30.00,8.50,7.40,2.51,2.23,168.00,109.00,19.00,15.00,8.90,8.90,27.40,27.40,NaN,NaN,233.00,233.00,4.00,3.40,136.00,134.00,14.10,14.10,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,63.00,13.00,22.73
1,246060,77.00,0,Caucasian,F,160.00,Floor,Floor,admit,Med-Surg ICU,0.93,70.20,NaN,108.00,203.01,0,0,NaN,9.00,0.56,145.00,120.00,36.90,0,46.00,33.00,145.00,35.10,NaN,1,12.70,95.00,31.00,118.00,72.00,120.00,38.00,32.00,12.00,100.00,70.00,159.00,67.00,36.30,35.10,1.60,1.60,0.50,0.50,11.00,9.00,8.60,8.00,0.71,0.56,145.00,128.00,27.00,26.00,11.30,11.10,36.90,36.10,1.30,1.30,557.00,487.00,4.20,3.80,145.00,145.00,23.30,12.70,37.00,37.00,7.45,7.45,51.00,51.00,0,0,0,0,0,0,64.00,5.00,27.42
2,276985,25.00,0,Caucasian,F,172.70,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,0.00,95.30,NaN,122.00,703.03,0,0,NaN,NaN,NaN,NaN,102.00,NaN,0,68.00,37.00,NaN,36.70,NaN,0,NaN,88.00,48.00,96.00,68.00,102.00,68.00,21.00,8.00,98.00,91.00,148.00,105.00,37.00,36.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,60.00,14.00,31.95


In [14]:
existing_cat_columns=[]
for col in df.columns:
    if df[col].dtype =='object':
        existing_cat_columns.append(col)
		
print(existing_cat_columns)

['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type']


In [15]:
from sklearn.preprocessing import LabelEncoder
for c in df.columns:
 if df[c].dtype == 'object':
     lbl = LabelEncoder()
     lbl.fit(list(df[c].values) + list(df[c].values))
     df[c] = lbl.transform(list(df[c].values))

In [16]:
df[existing_cat_columns].isnull().sum(axis=0)

ethnicity                0
gender                   0
hospital_admit_source    0
icu_admit_source         0
icu_stay_type            0
icu_type                 0
dtype: int64

In [18]:
binaryColumns = ['elective_surgery','apache_post_operative','intubated_apache','ventilated_apache','aids','cirrhosis','immunosuppression','leukemia','lymphoma','solid_tumor_with_metastasis']

In [19]:
for col in binaryColumns:
    df[col].fillna(df[col].mode()[0],inplace=True)

In [20]:
df[binaryColumns].isnull().sum(axis=0)

elective_surgery               0
apache_post_operative          0
intubated_apache               0
ventilated_apache              0
aids                           0
cirrhosis                      0
immunosuppression              0
leukemia                       0
lymphoma                       0
solid_tumor_with_metastasis    0
dtype: int64

In [21]:
existing_numerical_columns=[]
for col in df.columns:
    if df[col].dtype =='int32' or df[col].dtype =='float32' or df[col].dtype =='int64' or df[col].dtype =='float64':
        existing_numerical_columns.append(col)
for i in existing_numerical_columns:
    print(df[i].value_counts())

262144    1
186894    1
190984    1
188937    1
195082    1
         ..
171257    1
169208    1
191735    1
189686    1
266238    1
Name: encounter_id, Length: 140391, dtype: int64
67.00    3447
68.00    3293
71.00    3291
66.00    3217
72.00    3201
65.00    3146
63.00    3104
70.00    3096
73.00    3002
62.00    2980
69.00    2953
64.00    2933
74.00    2931
75.00    2907
61.00    2894
60.00    2869
59.00    2869
77.00    2836
76.00    2824
57.00    2796
78.00    2755
58.00    2711
79.00    2669
56.00    2666
55.00    2637
54.00    2531
80.00    2485
81.00    2464
82.00    2429
53.00    2429
83.00    2365
84.00    2328
52.00    2298
51.00    2119
85.00    2107
86.00    1985
50.00    1982
87.00    1859
49.00    1758
88.00    1623
48.00    1545
47.00    1489
89.00    1431
46.00    1425
45.00    1382
44.00    1187
43.00    1152
42.00    1106
41.00    991 
36.00    903 
40.00    899 
39.00    852 
38.00    834 
34.00    824 
35.00    820 
37.00    816 
33.00    760 
30.00    754 
32.00  

113.00    1131
122.00    1128
114.00    1123
112.00    1113
119.00    1097
          ... 
548.00    2   
610.00    2   
555.00    2   
541.00    1   
606.00    1   
Name: d1_glucose_max, Length: 539, dtype: int64
 97.00    2069
 96.00    2046
 98.00    2008
 95.00    1995
100.00    1986
          ... 
284.00    10  
292.00    3   
289.00    2   
290.00    1   
291.00    1   
Name: d1_glucose_min, Length: 262, dtype: int64
24.00    12841
25.00    12638
23.00    11624
26.00    11510
22.00    9549 
         ...  
32.40    1    
13.80    1    
18.80    1    
13.20    1    
14.80    1    
Name: d1_hco3_max, Length: 246, dtype: int64
23.00    11730
24.00    11710
25.00    10897
22.00    10297
26.00    9391 
         ...  
38.70    1    
36.10    1    
 7.90    1    
34.40    1    
10.70    1    
Name: d1_hco3_min, Length: 278, dtype: int64
11.60    2221
10.90    2175
11.90    2099
11.30    2098
11.20    2093
11.70    2069
11.50    2050
11.80    2043
11.10    2013
10.40    2006
12.20    2005


In [22]:
df[existing_numerical_columns].isnull().sum(axis=0)

encounter_id                   0    
age                            4988 
elective_surgery               0    
ethnicity                      0    
gender                         0    
height                         2378 
hospital_admit_source          0    
icu_admit_source               0    
icu_stay_type                  0    
icu_type                       0    
pre_icu_los_days               0    
weight                         4371 
albumin_apache                 84218
apache_2_diagnosis             1855 
apache_3j_diagnosis            973  
apache_post_operative          0    
arf_apache                     0    
bilirubin_apache               88870
bun_apache                     27472
creatinine_apache              26925
glucose_apache                 15906
heart_rate_apache              339  
hematocrit_apache              28872
intubated_apache               0    
map_apache                     472  
resprate_apache                872  
sodium_apache                  26512
t

In [23]:
nonBinaryColumns = [x for x in existing_numerical_columns if x not in binaryColumns]
print(nonBinaryColumns)

['encounter_id', 'age', 'ethnicity', 'gender', 'height', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'pre_icu_los_days', 'weight', 'albumin_apache', 'apache_2_diagnosis', 'apache_3j_diagnosis', 'arf_apache', 'bilirubin_apache', 'bun_apache', 'creatinine_apache', 'glucose_apache', 'heart_rate_apache', 'hematocrit_apache', 'map_apache', 'resprate_apache', 'sodium_apache', 'temp_apache', 'urineoutput_apache', 'wbc_apache', 'd1_diasbp_max', 'd1_diasbp_min', 'd1_heartrate_max', 'd1_heartrate_min', 'd1_mbp_max', 'd1_mbp_min', 'd1_resprate_max', 'd1_resprate_min', 'd1_spo2_max', 'd1_spo2_min', 'd1_sysbp_max', 'd1_sysbp_min', 'd1_temp_max', 'd1_temp_min', 'd1_albumin_max', 'd1_albumin_min', 'd1_bilirubin_max', 'd1_bilirubin_min', 'd1_bun_max', 'd1_bun_min', 'd1_calcium_max', 'd1_calcium_min', 'd1_creatinine_max', 'd1_creatinine_min', 'd1_glucose_max', 'd1_glucose_min', 'd1_hco3_max', 'd1_hco3_min', 'd1_hemaglobin_max', 'd1_hemaglobin_min', 'd1_hematocrit_max', 'd1

In [24]:
for i in nonBinaryColumns:
    df[i].fillna(df[i].mean(),inplace=True)

In [25]:
df[nonBinaryColumns].isnull().sum(axis=0)

encounter_id             0
age                      0
ethnicity                0
gender                   0
height                   0
hospital_admit_source    0
icu_admit_source         0
icu_stay_type            0
icu_type                 0
pre_icu_los_days         0
weight                   0
albumin_apache           0
apache_2_diagnosis       0
apache_3j_diagnosis      0
arf_apache               0
bilirubin_apache         0
bun_apache               0
creatinine_apache        0
glucose_apache           0
heart_rate_apache        0
hematocrit_apache        0
map_apache               0
resprate_apache          0
sodium_apache            0
temp_apache              0
urineoutput_apache       0
wbc_apache               0
d1_diasbp_max            0
d1_diasbp_min            0
d1_heartrate_max         0
d1_heartrate_min         0
d1_mbp_max               0
d1_mbp_min               0
d1_resprate_max          0
d1_resprate_min          0
d1_spo2_max              0
d1_spo2_min              0
d

In [26]:
df.shape

(140391, 88)

In [27]:
X = df[:len(df1)]
XPred = df[len(df1):].reset_index(drop=True)

In [28]:
###############################################################################
#                        3. Create train and test set                         #
###############################################################################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.50,random_state = 1000, stratify=y)

In [30]:
###############################################################################
#                              5. Good Ol' Classifiers                        #
###############################################################################
# Initializing Support Vector classifier
classifier1 = SVC(C = 50, degree = 1, gamma = "auto", kernel = "rbf", probability = True)
# Initializing Multi-layer perceptron  classifier
#classifier2 = MLPClassifier(activation = "relu", alpha = 0.1, hidden_layer_sizes = (10,10,10),
#                            learning_rate = "constant", max_iter = 2000, random_state = 1000)

# Initialing Nu Support Vector classifier
#classifier3 = NuSVC(degree = 1, kernel = "rbf", nu = 0.25, probability = True)

# Initializing Random Forest classifier
classifier4 = RandomForestClassifier(n_estimators = 500, criterion = "gini", max_depth = 10,
                                     max_features = "auto", min_samples_leaf = 0.005,
                                     min_samples_split = 0.005, n_jobs = -1, random_state = 1000)

In [33]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

layer_one_estimators = [
                        ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
                        ('knn_1', KNeighborsClassifier(n_neighbors=5))             
                       ]
layer_two_estimators = [
                        ('dt_2', DecisionTreeClassifier()),
                        ('rf_2', RandomForestClassifier(n_estimators=50, random_state=42)),
                       ]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=LogisticRegression())

# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5,stratify=y, random_state=42)
clf.fit(X_train, y_train).score(X_test, y_test)

0.8043762196714762

In [35]:
ypred=clf.predict(XPred)

In [44]:
len(ypred)

10234

In [45]:
import pandas
df_csv = pandas.DataFrame(data={"col1": df2['encounter_id'], "col2": ypred})
#df_csv.to_csv("./stacking.csv", sep=',',index=False)


In [49]:
df_csv[['col1','col2']].to_csv("submission_stacking.csv",index=False)

In [46]:
df_csv.head(3)

,col1,col2
0,144740,0
1,141990,0
2,142038,0


In [47]:
df_csv.nunique()

col1    10234
col2    2    
dtype: int64

In [32]:
###############################################################################
#                             6. Stacking Classifier                          #
###############################################################################
# Initializing the StackingCV classifier
sclf = StackingClassifier(classifiers = [classifier1, classifier4],
                            shuffle = False,
                            use_probas = True,
                            cv = 5,
                            meta_classifier = SVC(probability = True))

In [33]:
###############################################################################
#                       7. Putting classifiers in a dictionary                #
###############################################################################
# Create list to store classifiers
classifiers = {"SVC": classifier1,"MLP": classifier2,"NuSVC": classifier3,"RF": classifier4,"Stack": sclf}

classifiers = {"SVC": classifier1,RF": classifier4,"Stack": sclf}

In [ ]:
###############################################################################
#                               8. Train classifiers                          #
###############################################################################
# Train classifiers
for key in classifiers:
    # Get classifier
    classifier = classifiers[key]
    
    # Fit classifier
    classifier.fit(X_train, y_train)
        
    # Save fitted classifier
    classifiers[key] = classifier			   

In [ ]:
###############################################################################
#                              9. Making predictions                          #
###############################################################################
# Get results
results = pd.DataFrame()
for key in classifiers:
    # Make prediction on test set
    y_pred = classifiers[key].predict_proba(X_test)[:,1]
    
    # Save results in pandas dataframe object
    results[f"{key}"] = y_pred

# Add the test set to the results object
results["Target"] = y_test

In [ ]:
TARGET_COL = "diabetes_mellitus"

In [ ]:
df =pd.read_csv('/kaggle/input/widsdatathon2021/TrainingWiDS2021.csv')
test =pd.read_csv('/kaggle/input/widsdatathon2021/UnlabeledWiDS2021.csv')

In [ ]:
df.drop(df.columns[df.columns.str.contains('^h1')], axis=1, inplace=True)

In [ ]:
df.drop(['hospital_id'], axis=1, inplace=True)

In [ ]:
df.drop(df.columns[df.columns.str.contains('^Un')], axis=1, inplace=True)

In [ ]:
df.drop(df.columns[df.columns.str.contains('invasive')], axis=1, inplace=True)
df.drop(df.columns[df.columns.str.contains('noninvasive')], axis=1, inplace=True)

In [ ]:
cols = df.columns[df.isnull().mean()>0.7]

In [ ]:
print(cols)

In [ ]:
len(cols)

In [ ]:
df.drop(cols, axis=1, inplace=True)

In [ ]:
df.info()
df.value_counts()

In [ ]:
df.shape

In [ ]:
df.head(3)

In [ ]:
dfCorr = df.corr()

In [ ]:
corrlist = df.corr().unstack().sort_values().drop_duplicates()

In [ ]:
print(corrlist)

In [ ]:
kot = dfCorr[dfCorr>=0.99]

In [ ]:
plt.figure(figsize=(15,12))
sns.heatmap(kot,cmap="Greens")

In [ ]:
c = df.corr().abs()
s = c.unstack()
so = s.sort_values(kind="quicksort")


In [ ]:
print(so)

In [ ]:
print([c for c in df.columns if 7<df[c].nunique()<800])

In [ ]:
print([c for c in df.columns if (1<df[c].nunique()) & (df[c].dtype != np.number)& (df[c].dtype != int) ])

In [ ]:
categorical_cols =  ['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type']

In [ ]:
display(df[categorical_cols].dtypes)

In [ ]:
display(df[categorical_cols].isna().sum())

In [ ]:
df[categorical_cols] = df[categorical_cols].fillna("")

In [ ]:
display(df[categorical_cols].dtypes)

In [ ]:
display(df[categorical_cols].isna().sum())

In [ ]:
#for col in categorical_cols:
#     df[col] = pd.to_numeric(df[col], errors='coerce')
     

In [ ]:
#for col in categorical_cols:
#    df[col]=df[col].fillna(df[col].mode(),inplace=True)

In [ ]:
 df['ethnicity'] = pd.to_numeric(df['ethnicity'],errors='coerce')

In [ ]:
df['gender'] = pd.to_numeric(df['ethnicity'],errors='coerce')
df['hospital_admit_source'] = pd.to_numeric(df['hospital_admit_source'],errors='coerce')
df['icu_admit_source'] = pd.to_numeric(df['icu_admit_source'],errors='coerce')
df['icu_stay_type'] = pd.to_numeric(df['icu_stay_type'],errors='coerce')
df['icu_type'] = pd.to_numeric(df['icu_type'],errors='coerce')

In [ ]:
df.dtypes

In [ ]:
numerical_Columns=[]
for i in df.columns:
    if df[i].dtype=='int32' or df[i].dtype=='float32' or  df[i].dtype=='int64' or df[i].dtype=='float64':
        numerical_Columns.append(i)

In [ ]:
df.dtypes

In [ ]:
df[numerical_Columns] = df[numerical_Columns].fillna(0)

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

In [ ]:
df['ethnicity']=df['ethnicity'].fillna(0)

In [ ]:
df.isna().sum()

In [ ]:
df['gender']=df['gender'].fillna(0)
df['hospital_admit_source']=df['hospital_admit_source'].fillna(0)
df['icu_admit_source']=df['icu_admit_source'].fillna(0)
df['icu_stay_type']=df['icu_stay_type'].fillna(0)
df['icu_type']=df['icu_type'].fillna(0)


In [ ]:
df.isna().sum()

In [ ]:
X_train = df.drop([TARGET_COL],axis=1)
y_train = df[TARGET_COL]

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import LogisticRegression


In [ ]:
from sklearn.linear_model import (LinearRegression, Ridge, Lasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# create pipeline
model = LogisticRegression()
#rfe = RFECV(estimator=DecisionTreeClassifier())
#model = DecisionTreeClassifier()
#pipeline = Pipeline(steps=[('s',rfe),('m',model)])

In [ ]:
rfe = RFE(model, 20)
rfe = rfe.fit(X_train, y_train)

In [ ]:
# summarize the selection of the attributes
#print(rfe.support_)
#print(rfe.ranking_)

In [ ]:
# Feature Importance
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
# fit an Extra Trees model to the data
model = ExtraTreesClassifier()


In [ ]:
model.fit(X_train, y_train)
# display the relative importance of each attribute
print(model.feature_importances_)

In [ ]:
importance = model.feature_importances_

In [ ]:
(pd.Series(model.feature_importances_, index=X_train.columns)
   .nlargest(20)
   .plot(kind='barh')) 

In [ ]:
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
from matplotlib import pyplot
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
n_scores = cross_val_score(pipeline, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)

In [ ]:
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
print([c for c in df.columns if 7<df[c].nunique()<800])

In [ ]:
print([c for c in df.columns if (1<df[c].nunique()) & (df[c].dtype != np.number)& (df[c].dtype != int) ])

In [ ]:
categorical_cols =  ['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type']

In [ ]:
display(df[categorical_cols].dtypes)
display(df[categorical_cols].tail(3))
display(df[categorical_cols].isna().sum())

In [ ]:
df[categorical_cols] = df[categorical_cols].fillna("")

# same transformation for test data
test[categorical_cols] = test[categorical_cols].fillna("")

df[categorical_cols].isna().sum()

In [ ]:
for col in categorical_cols:
     df[col].fillna(df[col].mode()[0],inplace=True)

In [ ]:
display(df[categorical_cols].dtypes)

In [ ]:
#df['apache_2_diagnosis'] = pd.to_numeric(df['apache_2_diagnosis'], errors='coerce')

In [ ]:
#df['apache_2_diagnosis'] = df['apache_2_diagnosis'].fillna(0)

In [ ]:
#df.apache_3j_diagnosis.unique()

In [ ]:
display(df[categorical_cols].dtypes)

In [ ]:
#df['apache_3j_diagnosis']=df['apache_3j_diagnosis'].astype(int)#

In [ ]:
#display(df['apache_3j_diagnosis'].dtypes)

In [ ]:
#display(df[categorical_cols].dtypes)#

In [ ]:
#df['apache_2_diagnosis']=df['apache_2_diagnosis'].astype(int)

In [ ]:
display(df[categorical_cols].dtypes)

In [ ]:
display(df[categorical_cols].tail(3))

In [ ]:
display(df[categorical_cols].isna().sum())

In [ ]:
display(df[categorical_cols].isna().sum())

In [ ]:
df[categorical_cols] = df[categorical_cols].fillna("")

In [ ]:
test[categorical_cols] = test[categorical_cols].fillna("")
df[categorical_cols].isna().sum()
test[categorical_cols].isna().sum()

In [ ]:
X_train = df.drop([TARGET_COL],axis=1)
y_train = df[TARGET_COL]

In [ ]:
df[categorical_cols].info()

In [ ]:
df[categorical_cols].dtypes

In [ ]:
## catBoost Pool object
train_pool = Pool(data=X_train,label = y_train,cat_features=categorical_cols)
#                   baseline= X_train[""], ## 
#                   group_id = X_train['hospital_id']
                 

In [ ]:
model_basic = CatBoostClassifier(verbose=False,iterations=50)#,learning_rate=0.1, task_type="GPU",)

In [ ]:
model_basic.fit(train_pool, plot=True,silent=True)

In [ ]:
print(model_basic.get_best_score())

In [ ]:
### hyperparameter tuning example grid for catboost : 
grid = {'learning_rate': [0.04, 0.1],
        'depth': [7, 11],
#         'l2_leaf_reg': [1, 3,9],
#        "iterations": [500],
       "custom_metric":['Logloss', 'AUC']}

In [ ]:
model = CatBoostClassifier()

In [ ]:
## can also do randomized search - more efficient typically, especially for large search space - `randomized_search`
grid_search_result = model.grid_search(grid, 
                                       train_pool,
                                       plot=True,
                                       refit = True, #  refit best model on all data
                                      partition_random_seed=42)

In [ ]:
print(model.get_best_score())

In [ ]:
print("best model params: \n",grid_search_result["params"])

In [ ]:
feature_importances = model.get_feature_importance(train_pool)

In [ ]:
feature_names = X_train.columns

In [ ]:
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    if score > 0.05:
        print('{0}: {1:.2f}'.format(name, score))

In [ ]:
import shap

In [ ]:
shap.initjs()

In [ ]:
explainer = shap.TreeExplainer(model)

In [ ]:
shap_values = explainer.shap_values(train_pool)

In [ ]:
# visualize the training set predictions
# SHAP plots for all the data is very slow, so we'll only do it for a sample. Taking the head instead of a random sample is dangerous! 
shap.force_plot(explainer.expected_value,shap_values[0,:400], X_train.iloc[0,:400])

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, X_train)

In [ ]:
# test[TARGET_COL] = model.predict(test.drop([TARGET_COL],axis=1),prediction_type='Probability')[:,1]#

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, X_train)

In [ ]:
shap_values = shap.TreeExplainer(model).shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar")

In [ ]:
shap.dependence_plot('diabetes_mellitus', shap_values, X_train)